In [16]:
# import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt     
import seaborn as sns  
import missingno as msn   
import janitor
import re     
from dateutil import parser
# Install local package using "pip install -e . --config-setting editable_mode=compat"
import src  
from src.clean import load_data_and_dict       
from collections import Counter
import os
from dotenv import load_dotenv

In [17]:
# Load the west africa data 
load_dotenv() 

df = pd.read_excel(os.getenv('TBDATA_FILE_PATH'), sheet_name='PresumedTB -Program Grant')

In [18]:
def clean_column(val):
    if pd.isna(val):
        return val
    val = str(val).strip()
    try:
        parsed_date = parser.parse(val, fuzzy=False)
        return parsed_date.strftime('%Y-%m-%d')
    except (ValueError, TypeError):
        pass
    val = val.lower()
    if re.match(r"^\s*$|^_+$|^(n/a|na|null|none|unknown|\?)$", val, re.IGNORECASE):
        return pd.NA
    val = re.sub(r"\s+", "_", val)
    val = re.sub(r"[^a-z0-9_]", "", val)
    return val

def full_cleaning_pipeline(df):
    # Remove first part of column names and lowercase them
    df.columns = df.columns.str.split("_", n=1).str[-1].str.lower()
    
    # Combine age columns if they exist
    if 'ageyears' in df.columns and 'agemonths' in df.columns:
        df['age'] = df['ageyears'].combine_first(df['agemonths'] / 12)
        df = df.drop(columns=['ageyears', 'agemonths'])
    
    # Apply column cleaning
    df = df.apply(lambda col: col.map(clean_column) if col.dtype == 'object' else col)
    
    # Drop duplicates based on subjectid (lowercased)
    if 'subjectid' in df.columns:
        df = df.drop_duplicates(subset='subjectid')
    
    return df

# Apply pipeline
df = full_cleaning_pipeline(df)
display(df.head())



,dateseen,subjectid,batchno,adultindexcase,patientinitials,interviewer,sex,cough,productive,chestpain,wheeze,fever_chills,lossweight,difficultbreathing,nightsweats,sidepain,malaise_fatigue,lossappetite,haemoptysis,vomiting,numberepisode,pretbhistory,othsymptom,bcgdate,bcgscar,epiuptodate,pallor,splenomegaly,jaundice,hepatomegaly,wasting,oedema,clubbing,irritability,dyspnoea,deformityofspine,lethargy,abdominaldistension,spasticity,specifysites,lymphadenopathy,temperaturedc,pulsepm,otherclinicalsigns,respiratoryrate,subcostalrecession,dullness,dullnesswhere,crepitation,crepitationwhere,bronchialbreathing,bronchialbreathingwhere,rhonchi,rhonchiwhere,reducedbreathsounds,reducedbreathsoundswhere,other,otherwhere,weightkg,heightcm,of_circumferencecm,weightforage,weightforheight,heightforage,skinfoldthicknessmm,mua_circumferencemm,bmiforage,chestxraydone,chestxraydate,xrayquality,xraynormal,xrayabnormaldescribe,lobarconsolidation,lobarconsolidation_appa_r,lobarconsolidation_appa_l,lobarconsolidation_lat_uz,lobarconsolidation_lat_lz,bronchopneumonic,bronchopneumonic_appa_r,bronchopneumonic_appa_l,bronchopneumonic_lat_uz,bronchopneumonic_lat_lz,perihilaradenopathy,perihilaradenopathy_appa_r,perihilaradenopathy_appa_l,perihilaradenopathy_lat_uz,perihilaradenopathy_lat_lz,hyperinflation,hyperinflation_appa_r,hyperinflation_appa_l,hyperinflation_lat_uz,hyperinflation_lat_lz,interstitialchanges,interstitialchanges_appa_r,interstitialchanges_appa_l,interstitialchanges_lat_uz,interstitialchanges_lat_lz,nodularinfiltration,nodularinfiltration_appa_r,nodularinfiltration_appa_l,nodularinfiltration_lat_uz,nodularinfiltration_lat_lz,calcificationinthelung,calcificationinthelung_appa_r,calcificationinthelung_appa_l,calcificationinthelung_lat_uz,calcificationinthelung_lat_lz,fibrosis,fibrosis_appa_r,fibrosis_appa_l,fibrosis_lat_uz,fibrosis_lat_lz,cavity,cavity_appa_r,cavity_appa_l,cavity_lat_uz,cavity_lat_lz,volumeloss,volumeloss_appa_r,volumeloss_appa_l,volumeloss_lat_uz,volumeloss_lat_lz,perihilarinfiltration,perihilarinfiltration_appa_r,perihilarinfiltration_appa_l,perihilarinfiltration_lat_uz,perihilarinfiltration_lat_lz,paratrachealnodes,paratrachealnodes_appa_r,paratrachealnodes_appa_l,paratrachealnodes_lat_uz,paratrachealnodes_lat_lz,airwaycompression,airwaycompression_appa_r,airwaycompression_appa_l,airwaycompression_lat_uz,airwaycompression_lat_lz,pleuralfluid,pleuralfluid_appa_r,pleuralfluid_appa_l,pleuralfluid_lat_uz,pleuralfluid_lat_lz,trachealdeviation,softtissuechanges,mediastinaldeviation,spineabnormality,cardiacabnormality,otherbonyabnormality,otherbonyabnormality_specify,displaceddiaphragm,assessment,assessment_note,otherxraydiagnoses,sputumsmear,sputumdate,sputumtype,genexpertresult,cultureresult,othmycobacspecify,othmicrobacwhenpos,othmicrobacwhenposdays,skintestdone,skintestdate,skintestinduration,skintestdiametermm,diagnosis,eptbspecify,nottbdiagnosis,tbscore,antitbtreatment,treatmentplace,treatmentstartdate,treatmentid,otherplan,nextappointment,nextappointmentdate,entrydate,age
0,2012-02-14,r10023,0,11j105,fs,sb,female,cough__2_weeks,no,yes,no,yes,no,no,no,no,no,no,no,yes,NaN,NaN,NaN,NaT,yes,card_not_available,no,no,no,no,yes,no,no,no,no,no,no,no,no,NaN,no,36.5,88.0,<NA>,32.0,no,no,NaN,no,NaN,yes,left,yes,both,no,NaN,NaN,NaN,21.2,124.0,47.9,NaN,NaN,NaN,NaN,16.8,NaN,yes,2012-02-10,satisfactory,no,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,yes,yes,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,yes,no,no,no,no,no,NaN,no,abnormallikely_tb,NaN,<NA>,yes,2012-02-16,induced,not_done_at_all,negative,NaN,NaT,NaN,yes,2012-02-11,no,0.0,not_tb,NaN,lower_respiratory_tract_infection,7.0,no,NaN,NaT,NaN,for_3_monthly_symptom_screening_follow_up,NaN,NaT,2012-06-21 10:26:00,8.000000
1,2012-02-14,r10025,0,11j105,ml,sb,male,cough__2_weeks,yes,yes,no,no,yes,no,no,no,no,yes,no,n

In [19]:
# Inspect Missing Values
missing_values = df.isnull().sum()
missing_proportions = missing_values / len(df )
missing_df = pd.concat([missing_values, missing_proportions], axis=1)

missing_df.columns = ['Number of Missing Values', 'Proportion of Missing Values']
missing_df = missing_df.sort_values('Proportion of Missing Values', ascending=False)


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
#display(missing_df)

In [20]:
# Mapping of second dataset columns
column_mapping = {
    "subjectid": "patient_id",
    "dateseen": "date_particip",
    "sex": "gender",
    "age": "age",
    "cough": "cough",
    "productive": "productive",
    "chestpain": "chest_pain",
    "wheeze": "wheeze",
    "fever_chills": "fever_chills",
    "lossweight": "loss_weight",
    "difficultbreathing": "difficulty_breathing",
    "nightsweats": "night_sweats",
    "sidepain": "side_pain",
    "lossappetite": "loss_appetite",
    "haemoptysis": "haemoptysis",
    "vomiting": "vomiting",
    "pretbhistory": "pretb_history",  
    "pallor": "pallor",
    "splenomegaly": "splenomegaly",
    "jaundice": "jaundice",
    "hepatomegaly": "hepatomegaly",
    "wasting": "wasting",
    "dyspnoea": "dyspnoea",
    "lethargy": "lethargy",
    "abdominaldistension": "abd_distension_mass",
    "lymphadenopathy": "neck_lymph",
    "temperaturedc": "temperature",
    "weightkg": "weight",
    "heightcm": "height",
    "mua_circumferencemm": "mua_circumference",
    "otherclinicalsigns": "other_clinical_sign",
    "respiratoryrate": "respiratory_rate",
    "subcostalrecession": "subscostal_rec",
    "dullness": "dullness",
    "crepitation": "crepitation",
    "bronchialbreathing": "bronchial_breathing",
    "rhonchi": "rhonchi",
    "reducedbreathsounds": "reduced_breath_sounds",
    "other": "other_abnorm",
    "assessment": "xray_assessment",
    "otherxraydiagnoses": "other_xray",
    "sputumtype": "sputum_type",
    "genexpertresult": "genexpert_result",
    "cultureresult": "mgit_result",
    "othmycobacspecify": "other_mycobactera",
    "diagnosis": "diagnosis",
    "eptbspecify": "eptb_site",
    "nottbdiagnosis": "nottb_diagnosis",
    "antitbtreatment": "antitb_treatment"
}

# Select and rename columns
df = df[list(column_mapping.keys())].rename(columns=column_mapping)


In [21]:
df.columns

Index(['patient_id', 'date_particip', 'gender', 'age', 'cough', 'productive',
       'chest_pain', 'wheeze', 'fever_chills', 'loss_weight',
       'difficulty_breathing', 'night_sweats', 'side_pain', 'loss_appetite',
       'haemoptysis', 'vomiting', 'pretb_history', 'pallor', 'splenomegaly',
       'jaundice', 'hepatomegaly', 'wasting', 'dyspnoea', 'lethargy',
       'abd_distension_mass', 'neck_lymph', 'temperature', 'weight', 'height',
       'mua_circumference', 'other_clinical_sign', 'respiratory_rate',
       'subscostal_rec', 'dullness', 'crepitation', 'bronchial_breathing',
       'rhonchi', 'reduced_breath_sounds', 'other_abnorm', 'xray_assessment',
       'other_xray', 'sputum_type', 'genexpert_result', 'mgit_result',
       'other_mycobactera', 'diagnosis', 'eptb_site', 'nottb_diagnosis',
       'antitb_treatment'],
      dtype='object')

`There are 8 columns that don't match: "malaise_fatigue", "date_previous_tb", "abnorm_site", "specimen_type", "spec_result", "histology_result", "hiv_serology", "country"`

In [22]:
# Inspect Missing Values
missing_values = df.isnull().sum()
missing_proportions = missing_values / len(df )
missing_df = pd.concat([missing_values, missing_proportions], axis=1)

missing_df.columns = ['Number of Missing Values', 'Proportion of Missing Values']
missing_df = missing_df.sort_values('Proportion of Missing Values', ascending=False)


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
display(missing_df)

,Number of Missing Values,Proportion of Missing Values
other_mycobactera,924,1.000000
eptb_site,911,0.985931
other_abnorm,864,0.935065
mgit_result,819,0.886364
other_clinical_sign,702,0.759740
other_xray,592,0.640693
nottb_diagnosis,209,0.226190
genexpert_result,148,0.160173
pretb_history,70,0.075758
sputum_type,67,0.072511


In [23]:
df.shape

(924, 49)